# Месячный отчет

Постановка задачи: Из таблица представленной в Google sheets сформировать отчет по заданной форме в совместную беседу во ВКонтакте.



## Загрузка данных
Загружаем dataframe из Google sheets

В таблице Google нажмите файл> опубликовать в Интернете. Затем выберите то, что вам нужно опубликовать, и выберите формат экспорта .csv. У вас будет ссылка, скопитруйте ее и вставьте в pathtocsv

https://question-it.com/questions/2560054/zagruzka-obschej-tablitsy-google-v-pandas


In [146]:
import pandas as pd

pathtocsv = r'https://docs.google.com/spreadsheets/d/e/2PACX-1vQJXb0sEZuwsdRUZ-rGWRMJaEwHgbJ5wZJKeVje9U52GQDp1hNd3OVsNybCuJh7xw-oc2FLF9AJBTeN/pub?gid=37&single=true&output=csv'
df = pd.read_csv(pathtocsv)
df.head()

,Адрес,Дата,Сумма,Направление,Категория,Контрагент,Подтверждено,Заметки,Долг
0,ДК1,NaN,"-21 000,00 р.",расход,аренда,Киршин,да,Аренда,NaN
1,ДК1,NaN,"-5 500,00 р.",расход,аренда,Киршин,да,Комуналка+ТБО,NaN
2,NaN,NaN,"-6 000,00 р.",расход,зп,Костя,да,топка печи (окт-апр) за сент и окт,NaN
3,NaN,NaN,"-2 000,00 р.",расход,ГСМ,Костя,да,бензин на месяц,NaN
4,NaN,NaN,"-500,00 р.",расход,Связь,Костя,да,телефон на месяц,NaN


## Формирование отчета
Вычисляем нужные показатели для отчета из Pandas датафрейма и приводим в требуемый вид

### Вычисление основных показателей

In [147]:
df['Сумма'] = df['Сумма'].apply(lambda x: str(x).replace(' ', ''))
df['Сумма'] = df['Сумма'].apply(lambda x: str(x).replace('р.', ''))
df['Сумма'] = df['Сумма'].apply(lambda x: str(x).replace(u'\xa0', ''))
df['Сумма'] = df['Сумма'].apply(lambda x: str(x).replace(',', '.'))
df['Сумма'] = df['Сумма'].astype(float)
profit_tmp = df['Сумма'].sum()
profit_tmp

-32650.0

In [148]:
df_debts = df.loc[df['Долг'] == 'сергей']
df_debts

,Адрес,Дата,Сумма,Направление,Категория,Контрагент,Подтверждено,Заметки,Долг
12,NaN,09.10.2021,-400.0,расход,улучшения,озон,да,держатель отверок в дк2,сергей
22,NaN,17.10.2021,-2100.0,расход,улучшения,КрепежСервис,да,и ижинструмент,сергей
28,NaN,23.10.2021,-550.0,расход,амортизация,Строительный выгон,да,пена+пистолет,сергей
35,NaN,21.11.21,-1050.0,расход,амортизация,резинотехника,да,шланг 15м,сергей
36,NaN,21.11.21,-50.0,расход,амортизация,КрепежСервис,да,хомуты,сергей


In [149]:
# Долги ДК -> ОСА
debt_dk_to_osa = df_debts[df_debts['Категория'] == 'амортизация']['Сумма'].sum()
debt_dk_to_osa

-1650.0

In [150]:
# Все улучшения за месяц
upgrade_all = df[df['Категория'] == 'улучшения']['Сумма'].sum()
upgrade_all

-2500.0

In [151]:
# Прибыль/убыток с учетом трат на улучшения
profit = profit_tmp - upgrade_all

In [152]:
# Улучшения за счет ОСА в месяц
upgrade_osa = df_debts[df_debts['Категория'] == 'улучшения']['Сумма'].sum()
upgrade_osa

-2500.0

In [153]:
# Улучшения за счет ЦКБ в месяц
upgrade_ckb = upgrade_all - upgrade_osa
upgrade_ckb

0.0

In [154]:
# Улучшения 50/50: ЦКБ -> ОСА
upgrade_debts = upgrade_osa / 2 - (upgrade_osa / 2 % 50) - upgrade_ckb / 2 - (
    upgrade_ckb / 2 % 50)
upgrade_debts

-1250.0

In [155]:
# Итог по улучшениям
if upgrade_osa <= upgrade_ckb:
    print('Улучшения 50/50: ЦКБ -> ОСА', -upgrade_debts)
else:
    print('Улучшения 50/50: ОСА -> ЦКБ', upgrade_debts)

Улучшения 50/50: ЦКБ -> ОСА 1250.0


### Расчет прибыли/убытка и аренды

In [156]:
rent = 10000
coef = 0.7


def profit_calculation(profit, rent=rent, coef=coef):
    """
    Доход ОСА и ЦКБ
    """
    if profit >= 0:
        if (profit * coef) % 50 == 0:
            ckb = profit * coef // 50 * 50
            osa = profit - profit * coef // 50 * 50
        else:
            ckb = (profit * coef + 50) // 50 * 50
            osa = profit - (profit * coef + 50) // 50 * 50
    elif -rent <= profit < 0:
        if (profit * coef) % 50 == 0:
            # Аренду убираем, оставляем только прибыль. Аренда rent.
            ckb = ((profit + rent) * coef) // 50 * 50
            osa = (profit + rent) - (profit + rent) * coef // 50 * 50
        else:
            ckb = ((profit + rent) * coef + 50) // 50 * 50
            osa = (profit + rent) - ((profit + rent) * coef + 50) // 50 * 50
    else:
        ckb = 0
        osa = 0
    return ckb, osa

In [157]:
assert profit_calculation(profit=0, rent=rent, coef = coef) == (0, 0)
assert profit_calculation(profit=-30000, rent=rent, coef = coef) == (0, 0)
assert profit_calculation(profit=-31000, rent=rent, coef = coef) == (0, 0)
assert profit_calculation(profit=-10000, rent=rent, coef = coef) == (0, 0)

assert profit_calculation(profit=10000, rent=rent, coef = coef) == (7000, 3000)
assert profit_calculation(profit=10050, rent=rent, coef = coef) == (7050, 3000)

assert profit_calculation(profit=-5000, rent=rent, coef = coef) == (3500, 1500)
assert profit_calculation(profit=-5050, rent=rent, coef = coef) == (3500, 1450)

In [158]:
def rent_calculation(profit, rent=rent):
    if profit > 0:
        return rent / 2
    else:
        return 0

In [159]:
assert rent_calculation(profit=0, rent=rent) == 0
assert rent_calculation(profit=-500, rent=rent) == 0
assert rent_calculation(profit=500, rent=rent) == 5000

### Формирование структуры отчета

In [160]:
from datetime import datetime as dt
from datetime import timedelta
report_period = dt.now().date()
report_period = report_period-timedelta(days=report_period.day)
text = ''
text += f"Итоги {report_period.strftime('%b %Y')}г.:\n"
text += f'Аренда: {rent_calculation(profit)}р. ЦКБ, {rent_calculation(profit)}р. ОСА\n'
text += f"Чистая прибыль/убыток: {profit}р. ЦКБ {profit_calculation(profit)[0]}р. ОСА {profit_calculation(profit)[1]}р.\n-----\n"
text += f"Долги ДК -> ОСА {-debt_dk_to_osa}р.\n"

for i, row in df_debts[~(df_debts['Категория']=='улучшения')].iterrows():
    if df_debts[~(df_debts['Категория']=='улучшения')].shape[0]!=0:
        row_without_line_break = str(row.values).replace('\n', '')
        text += f"{i} {row_without_line_break}\n"

text +='-----\n'  
# Итог по улучшениям
upgrade=0
if upgrade_osa <= upgrade_ckb:
   text += f'Улучшения 50/50: ЦКБ -> ОСА {-upgrade_debts}р.\n'
   upgrade=-upgrade_debts
else:
    text += f'Улучшения 50/50: ОСА -> ЦКБ {upgrade_debts}р.\n'
    upgrade=upgrade_debts

for i, row in df_debts[(df_debts['Категория']=='улучшения')].iterrows():
    if df_debts[(df_debts['Категория']=='улучшения')].shape[0]!=0:
        row_without_line_break = str(row.values).replace('\n', '')
        text += f"{i} {row_without_line_break}\n"

text +='-----\n'  
text += f"Итого с ЦКБ -> ОСА {rent_calculation(profit)+profit_calculation(profit)[1]-debt_dk_to_osa+upgrade}р."
text_list = text.split('\n')
text_list

['Итоги Oct 2021г.:',
 'Аренда: 0р. ЦКБ, 0р. ОСА',
 'Чистая прибыль/убыток: -30150.0р. ЦКБ 0р. ОСА 0р.',
 '-----',
 'Долги ДК -> ОСА 1650.0р.',
 "28 [nan '23.10.2021' -550.0 'расход' 'амортизация' 'Строительный выгон' 'да' 'пена+пистолет' 'сергей']",
 "35 [nan '21.11.21' -1050.0 'расход' 'амортизация' 'резинотехника' 'да' 'шланг 15м' 'сергей']",
 "36 [nan '21.11.21' -50.0 'расход' 'амортизация' 'КрепежСервис' 'да' 'хомуты' 'сергей']",
 '-----',
 'Улучшения 50/50: ЦКБ -> ОСА 1250.0р.',
 "12 [nan '09.10.2021' -400.0 'расход' 'улучшения' 'озон' 'да' 'держатель отверок в дк2' 'сергей']",
 "22 [nan '17.10.2021' -2100.0 'расход' 'улучшения' 'КрепежСервис' 'да' 'и ижинструмент' 'сергей']",
 '-----',
 'Итого с ЦКБ -> ОСА 2900.0р.']

## Отправка отчета в ВК
Отправка с помощью библиотеки selenium

In [161]:
!pip install selenium

In [162]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

LOGIN = ''
PASSWORD = ''

driver = webdriver.Chrome('/home/grey/documents/pet_project/chromedriver')
driver.get("http://www.vk.com")

login = driver.find_element_by_id("index_email")
login.clear()
login.send_keys(LOGIN)

pswd = driver.find_element_by_id("index_pass")
pswd.clear()
pswd.send_keys(PASSWORD)

btm_signin = driver.find_element_by_class_name('index_login_button.flat_button.button_big_text')
btm_signin.click()
time.sleep(3)

msg = driver.find_element(By.XPATH, '//span[text()="Мессенджер"]')
msg.click()
time.sleep(3)

talk_dk = driver.find_element(By.XPATH, '//span[text()="Доктор кузов"]')
talk_dk.click()
time.sleep(3)

entry_field = driver.find_element_by_class_name('im_editable.im-chat-input--text._im_text')


entry_field.clear()
for el in text_list:
    entry_field.send_keys(el)
    btm_send = driver.find_element_by_class_name('im-send-btn.im-chat-input--send._im_send.im-send-btn_send')
    time.sleep(0.4)
    btm_send.click()

driver.close()

<ipython-input-162-6def35b3b8db>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/grey/documents/pet_project/chromedriver')
<ipython-input-162-6def35b3b8db>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  login = driver.find_element_by_id("index_email")
<ipython-input-162-6def35b3b8db>:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pswd = driver.find_element_by_id("index_pass")
<ipython-input-162-6def35b3b8db>:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  btm_signin = driver.find_element_by_class_name('index_login_button.flat_button.button_big_text')
<ipython-input-162-6def35b3b8db>:32: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  entry_field = driver.find_element_by_class_name('im_editable.

## Иное
Изначально задумка была сделать отправку в ВК используя API функции. План провалился - либо ума не хватило, либо действительно такие отправки заколотили гвоздями.

In [163]:
# import requests
# import random

# MESSAGES_SEND_REQUEST = 'https://api.vk.com/method/messages.send'

# with open('token.txt', 'r') as file_object:
#     token_group = file_object.read().strip()
# VERSION = '5.131'

# rand = random.randint(0, 2**32)
# params = {
#     'access_token': token_group,
#     'user_id': 10398721,
#     'random_id': rand,
#     'message': text,
#     'v': VERSION,
# }

# response = requests.post(MESSAGES_SEND_REQUEST, params)
# print(response)
# response.text